In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Hard spheres simulation (Lennard-Jones fluid)

In [1]:
from simtk import openmm as mm
from simtk import unit
from simtk.openmm import app
import numpy as np

In [2]:
n_particles = 10
mass = 50.0 * unit.amu
sigma = 3.4 * unit.angstroms
epsilon = 0.238 * unit.kilocalories_per_mole
charge = 0.0 * unit.elementary_charge

In [3]:
system = mm.System()

nb = mm.NonbondedForce()
nb.setNonbondedMethod(mm.NonbondedForce.CutoffPeriodic)
nb.setCutoffDistance(2.0*sigma)
nb.setUseDispersionCorrection(True) #

for particle_index in range(n_particles):
    system.addParticle(mass)
    nb.addParticle(charge, sigma, epsilon)

_ = system.addForce(nb)

In [4]:
force = mm.CustomExternalForce('k * (z^2)')
force.addGlobalParameter('k', 100)
for particle_index in range(n_particles):
    force.addParticle(particle_index, [])

_ = system.addForce(force)

In [5]:
box_x = 15.0 * unit.angstroms
box_y = 15.0 * unit.angstroms
box_z = 15.0 * unit.angstroms

v1 = [box_x, 0.0*unit.angstroms, 0.0*unit.angstroms]
v2 = [0.0*unit.angstroms, box_y, 0.0*unit.angstroms]
v3 = [0.0*unit.angstroms, 0.0*unit.angstroms, box_z]

system.setDefaultPeriodicBoxVectors(v1, v2, v3)

In [6]:
coordinates = np.zeros([n_particles, 3]) * unit.angstroms

random_generator = np.random.default_rng()
coordinates[:,0] = random_generator.uniform(0.0, 1.0, n_particles) * box_x
coordinates[:,1] = random_generator.uniform(0.0, 1.0, n_particles) * box_y

In [7]:
integrator = mm.LangevinIntegrator(300 * unit.kelvin, 1.0 / unit.picosecond, 0.002 * unit.picoseconds)

In [8]:
platform = mm.Platform.getPlatformByName('CUDA')

In [9]:
context = mm.Context(system, integrator, platform)

In [10]:
context.setPositions(coordinates)
context.setVelocitiesToTemperature(300*unit.kelvin)

In [11]:
state = context.getState(getEnergy=True)
state.getPotentialEnergy()

Quantity(value=5.2787222516428125e+20, unit=kilojoule/mole)

In [12]:
for l in np.linspace(0.01, 1.0, num=100, endpoint=True):
    
    for ii in range(n_particles):
        nb.setParticleParameters(ii, charge, sigma*l, epsilon)
    
    mm.LocalEnergyMinimizer_minimize(context)
    integrator.step(1000)

In [13]:
state = context.getState(getEnergy=True)
state.getPotentialEnergy()

Quantity(value=15147967971326.791, unit=kilojoule/mole)

In [14]:
state = context.getState(getPositions=True)
coordinates = state.getPositions()
print(coordinates)

[Vec3(x=-1591271.755859375, y=-1637896.92578125, z=-936250.890625), Vec3(x=-5487533.073608398, y=-448028.173828125, z=-734882.6953125), Vec3(x=-3468840.14453125, y=1773393.5, z=530558.6171875), Vec3(x=-3761064.8173828125, y=-2159736.40625, z=-509551.6640625), Vec3(x=1784624.34375, y=-17177.218872070312, z=-519989.71875), Vec3(x=-5231005.15625, y=680032.9536132812, z=1007115.84375), Vec3(x=-2546463.4375, y=-1579928.55859375, z=646945.28125), Vec3(x=282047.52734375, y=-3314716.10546875, z=602017.73046875), Vec3(x=-1582077.09375, y=-8751303.6875, z=870364.90625), Vec3(x=1483718.796875, y=-4728808.9375, z=-1276794.4375)] nm


# Building a leaflet

In [ ]:
import openmembrane as om

In [ ]:
box_x = 2.7
box_y = 2.7
radii=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.3, 0.3, 0.3, 0.3, 0.4, 0.4, 0.4, 0.4, 0.4]

In [ ]:
leaflet = om.build_leaflet(box_x, box_y, radii)

In [ ]:
leaflet

In [ ]:
fig, ax = plt.subplots()

ax.scatter(leaflet[:,0], leaflet[:,1], s=1)
ax.set_aspect('equal')
ax.set_xlim([0.0, box_x])
ax.set_ylim([0.0, box_y])

for ii in range(len(radii)):
    radius = radii[ii]
    x = leaflet[ii,0]
    y = leaflet[ii,1]
    new_circle = plt.Circle((x, y), radius, fill=False)
    ax.add_artist(new_circle)

plt.show()